In [ ]:
%pip install numpy
%pip install bs4
%pip install scikit-optimize
%pip install line_profiler

In [ ]:
import os
import subprocess

# fabric_dir = "../fabric-samples"
# caliper_dir = "../caliper-benchmarks"


# if os.path.isdir(fabric_dir):
#     print("Directory exists")
# else:
#     subprocess.run("cd ../ && git clone https://github.com/hyperledger/fabric-samples.git", shell=True)

# if os.path.dirname(caliper_dir):
#     print("Directory exists")
# else:
#     subprocess.run("cd ../ && git clone https://github.com/hyperledger-caliper/caliper-benchmarks.git", shell=True)

In [ ]:
# subprocess.run("cd ../ && npm install --only=prod @hyperledger/caliper-cli", shell=True)
# subprocess.run("cd ../ && npx caliper bind --caliper-bind-sut fabric:2.4", shell=True)

In [ ]:
import yaml

def get_config(path: str):
    with open(path, 'r') as f:
        return yaml.safe_load(f)

In [ ]:
# !cd ../caliper-benchmarks/ && npx caliper launch manager --caliper-workspace ./ --caliper-networkconfig networks/fabric/test-network.yaml --caliper-benchconfig benchmarks/samples/fabric/fabcar/config.yaml --caliper-flow-only-test --caliper-fabric-gateway-enabled

In [ ]:
import json
from bs4 import BeautifulSoup

def convert_fabric_html_to_json(html_file, output_json_file):
    try:
        with open(html_file, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
        
        summary = soup.find("div", {"id": "summary"}).text.strip() if soup.find("div", {"id": "summary"}) else "No summary found"
        metrics = {}
        
        table = soup.find("table")
        if table:
            headers = [th.text.strip() for th in table.find_all("th")]
            rows = table.find_all("tr")[1:]
            for row in rows:
                cells = row.find_all("td")
                key = cells[0].text.strip()
                values = {headers[i]: cells[i].text.strip() for i in range(1, len(cells))}
                metrics[key] = values
        
        report_data = {
            "summary": summary,
            "metrics": metrics
        }
        
        with open(output_json_file, "w", encoding="utf-8") as json_file:
            json.dump(report_data, json_file, indent=4)
        
        print(f"JSON report successfully created: {output_json_file}")
    except Exception as e:
        print(f"Error during conversion: {e}")


In [ ]:
def calculate_average_tps(json_file):
    try:
        json_data = None
        with open(json_file, "r", encoding="utf-8") as file:
            json_data = json.load(file)
        tps_values = []
        for _, metrics in json_data["metrics"].items():
            tps_value = float(metrics.get("Throughput (TPS)", 0))
            tps_values.append(tps_value)
        
        if not tps_values:
            print("No TPS values found.")
            return None
        
        average_tps = sum(tps_values) / len(tps_values)
        return average_tps

    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:
# convert_fabric_html_to_json('../caliper-benchmarks/report.html', 'report.json')

In [ ]:
# avg_tps = calculate_average_tps('report.json')
# avg_tps

In [ ]:
import subprocess

# assumes binding is done to fabric 2.4
def objective():
    # start network
    network_starter = 'cd ../fabric-samples/test-network && ./network.sh up createChannel -s couchdb'
    subprocess.run(network_starter, shell=True)

    #deploy chaincode
    chaincode_deployer = 'cd ../fabric-samples/test-network && ./network.sh deployCC -ccn fabcar -ccp ../../caliper-benchmarks/src/fabric/samples/fabcar/go -ccl go'
    subprocess.run(chaincode_deployer, shell=True)

    # execute benchmark
    command = "cd ../caliper-benchmarks/ && npx caliper launch manager --caliper-workspace ./ --caliper-networkconfig networks/fabric/test-network.yaml --caliper-benchconfig benchmarks/samples/fabric/fabcar/config.yaml --caliper-flow-only-test --caliper-fabric-gateway-enabled"
    subprocess.run(command, shell=True)

    # process output
    convert_fabric_html_to_json('../caliper-benchmarks/report.html', 'report.json')
    avg_tps = calculate_average_tps('report.json')

    # shut down network
    shutter = 'cd ../fabric-samples/test-network && ./network.sh down'
    subprocess.run(shutter, shell=True)
    return avg_tps


In [ ]:
# score = objective()

In [ ]:
import numpy as np
from skopt import Optimizer
from skopt.space import Real, Integer
from skopt.utils import use_named_args

def fill_config(conf, next_point, sample_params):
    for i in range(len(sample_params)):
        internal_path = sample_params[i].name.split('.')
        parts = internal_path[-1].split('|')
        assert(len(parts) > 0 and len(parts) <= 2)
        internal_path[-1] = parts[0]

        assert(len(internal_path) > 0)
        link = conf[internal_path[0]]
        for j in range(1, len(internal_path)):
            link = link[internal_path[j]]
        # print(link)
        if len(parts) == 1:
            link = next_point[i]
        else:
            link = str(int(next_point[i])) + parts[1]
        pass

In [ ]:
CONFIG_PATH = '/home/admin1/hse/blockchain-benchmarks/fabric-samples/test-network/configtx/configtx.yaml'
conf = get_config(str(CONFIG_PATH))
# assert(False, "Config path")

# TODO: automate config

sample_params = [
    Integer(1, 19, name="Orderer.BatchSize.MaxMessageCount|"), # follow style of conf
    Integer(1, 49, name="Orderer.BatchSize.AbsoluteMaxBytes|MB"), # 49 is max recommended
    Integer(1, 2048, name="Orderer.BatchSize.PreferredMaxBytes|KB"),
    Integer(1, 30, name="Orderer.BatchTimeout|s"),
    Integer(1, 999, name="Profiles.ChannelUsingRaft.Orderer.EtcdRaft.Options.TickInterval|ms"),
    Integer(1, 100, name="Profiles.ChannelUsingRaft.Orderer.EtcdRaft.Options.ElectionTick"),
    Integer(1, 10, name="Profiles.ChannelUsingRaft.Orderer.EtcdRaft.Options.HeartbeatTick"),
    Integer(1, 50, name="Profiles.ChannelUsingRaft.Orderer.EtcdRaft.Options.MaxInflightBlocks"),
    Integer(1, 128, name="Profiles.ChannelUsingRaft.Orderer.EtcdRaft.Options.SnapshotIntervalSize|MB"),
]

# TODO: значимость параметров -- SHAP-score

# TODO: orderer.type == BFT

# обучение с привилегированными данными
# data fusion
# априорное распределение модели

optimizer = Optimizer(
    dimensions=sample_params,
    base_estimator="GP",
    n_initial_points=10,
    acq_func="gp_hedge",
    acq_optimizer="auto",
)

NUM_EPOCHS = 2
# learning

def fit(sample_params, optimizer, conf, num_epochs):

    scores = []
    optimals = [] # TODO: file

    for i in range(NUM_EPOCHS): # tqdm
        next_point = optimizer.ask()

        if next_point is None:
            print("No more points to evaluate.")
            break

        # fill config with better(?) params
        fill_config(conf, next_point, sample_params)

        # save config
        #TODO: why do it
        # with open(CONFIG_PATH, 'w') as f:
        #     yaml.dump(conf, f)

        # mop
        print(f"Next point to evaluate: {next_point}")
        optimals.append(next_point)
        value = objective() # 10 objectives for snr
        scores.append(value)
        print(f"Score: {value}\n")
        optimizer.tell(next_point, -value)



In [ ]:
from line_profiler import LineProfiler

profiler = LineProfiler()
profiler.add_function(fit)
profiler.enable()
fit(sample_params, optimizer, conf, NUM_EPOCHS)
profiler.disable()
profiler.print_stats()

In [ ]:
# import matplotlib.pyplot as plt


# plt.figure(figsize=(8, 5))
# plt.plot(scores, marker='o', linestyle='-', color='b', label='scores')

# plt.title("Graph of ml", fontsize=14)

# plt.xlabel("Index", fontsize=12)
# plt.ylabel("Value", fontsize=12)
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# plt.figure(figsize=(8, 5))
# plt.plot(optimals, marker='o', linestyle='-', color='b', label='scores')

# plt.title("Graph of ml", fontsize=14)
# plt.xlabel("Index", fontsize=12)
# plt.ylabel("Value", fontsize=12)
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# import numpy as np

# def snr(scores_global):
#     for scores in scores_global:
#         mean_score = np.mean(scores)
#         std_dev = np.std(scores, ddof=1)

#         if std_dev == 0:
#             return float('inf')  # Return infinity if no variation

#         print(mean_score / std_dev)

In [14]:
import numpy as np
arr = []
with open("scores_snr.txt") as f:
    for line in f.readlines():
        line = line[1:-2]
        for x in line.split(", "):
            arr.append(float(x))

noise_value = np.std(arr, ddof=1)
mean_score = np.mean(arr)
relative_noise = (noise_value / mean_score) * 100

relative_noise

6.7622677397895234

In [15]:
# arr = arr[0:-2]
# noise_value = np.std(arr, ddof=1)
# mean_score = np.mean(arr)
# relative_noise = (noise_value / mean_score) * 100 

# relative_noise

In [16]:
signal = np.mean([330.525, 321.925])
snr = signal / noise_value
snr

2.272535542675445